In [1]:
import copy
from IPython.display import display, Image
import logging
import math
import numpy
import os
import pandas
import pdb
import plotly
import pprint
import pyarrow
import pyarrow.parquet as pq
import random
import six
import sys
import tensorflow as tf
import time

import CS230

plotly.offline.init_notebook_mode(connected=True)

In [2]:
logger = CS230.common.get_logger(level=logging.INFO)

In [3]:
file_paths = CS230.data.get_all_file_paths()

01-26-41 | INFO | num files: 22


# data summary

In [4]:
file_path = file_paths[0]

df = CS230.data.load(file_path)
df = CS230.data.stride_rows(df, stride=10)

In [5]:
columns = list(df.keys())
columns.sort()
columns_formatted = ''
for x in columns:
    columns_formatted += '- ' + x + '\n'
#print(output)
print(columns)

['HDOP', 'PDOP', 'PPS', 'altitude', 'axCG', 'ayCG', 'azCG', 'brake', 'chassisAccelFL', 'chassisAccelFR', 'chassisAccelRL', 'chassisAccelRR', 'clutch', 'deflectionFL', 'deflectionFR', 'deflectionRL', 'deflectionRR', 'distance', 'engineSpeed', 'gpsOrientMode', 'gpsPosMode', 'gpsTime', 'gpsVelMode', 'handwheelAngle', 'horizontalSpeed', 'latitude', 'longitude', 'numSVsTracked', 'orientAccuracy_heading', 'orientAccuracy_pitch', 'orientAccuracy_roll', 'pitchAngle', 'pitchRate', 'posAccuracy_down', 'posAccuracy_east', 'posAccuracy_north', 'rollAngle', 'rollRate', 'sideSlip', 'throttle', 'time', 'vEast', 'vNorth', 'vUp', 'velAccuracy_down', 'velAccuracy_east', 'velAccuracy_north', 'vxCG', 'vyCG', 'vzCG', 'wheelAccelFL', 'wheelAccelFR', 'wheelAccelRL', 'wheelAccelRR', 'yawAngle', 'yawRate']


In [6]:
df.head()

,time,handwheelAngle,throttle,brake,clutch,engineSpeed,PPS,gpsTime,latitude,longitude,...,chassisAccelRL,chassisAccelRR,deflectionFL,deflectionFR,deflectionRL,deflectionRR,wheelAccelFL,wheelAccelFR,wheelAccelRL,wheelAccelRR
0,0.009,29.4,0.7,0.5,13.2,204,5.0,61377.09,NaN,NaN,...,-9.663,-9.712,-0.002,-0.003,0.003,0.001,9.91,9.71,9.61,10.07
1,0.019,29.4,0.7,0.0,13.2,204,5.0,61377.10,NaN,NaN,...,-9.633,-9.712,-0.002,-0.003,0.003,0.001,9.81,9.81,9.81,10.14
2,0.029,29.4,0.7,0.0,12.7,204,5.0,61377.11,NaN,NaN,...,-9.633,-9.712,-0.002,-0.003,0.003,0.001,10.10,9.81,9.61,10.09
3,0.039,29.4,0.7,0.0,12.7,204,5.0,61377.12,NaN,NaN,...,-9.624,-9.712,-0.002,-0.003,0.003,0.001,10.20,9.61,9.71,9.96
4,0.049,29.4,0.7,0.0,12.7,204,5.0,61377.13,NaN,NaN,...,-9.663,-9.614,-0.002,-0.003,0.003,0.001,10.10,10.01,9.91,10.08


In [7]:
df = CS230.data.add_derivatives(df, stride=1)
df = CS230.data.clean_discontinuities(df, stride=1)

In [8]:
df.head()

,time,handwheelAngle,throttle,brake,clutch,engineSpeed,PPS,gpsTime,latitude,longitude,...,deriv_rollAngle,deriv_rollRate,deriv_vxCG,deriv_vyCG,deriv_wheelAccelFL,deriv_wheelAccelFR,deriv_wheelAccelRL,deriv_wheelAccelRR,deriv_yawAngle,deriv_yawRate
0,0.009,29.4,0.7,0.5,13.2,204,5.0,61377.09,NaN,NaN,...,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.00
1,0.019,29.4,0.7,0.0,13.2,204,5.0,61377.10,NaN,NaN,...,0.0,1.19,0.0,-0.0,-0.10,0.1,0.2,0.07,0.0,-0.64
2,0.029,29.4,0.7,0.0,12.7,204,5.0,61377.11,NaN,NaN,...,0.0,-0.76,0.0,0.0,0.29,0.0,-0.2,-0.05,0.0,-0.06
3,0.039,29.4,0.7,0.0,12.7,204,5.0,61377.12,NaN,NaN,...,0.0,0.30,-0.0,0.0,0.10,-0.2,0.1,-0.13,0.0,0.81
4,0.049,29.4,0.7,0.0,12.7,204,5.0,61377.13,NaN,NaN,...,0.0,-0.26,0.0,0.0,-0.10,0.4,0.2,0.12,0.0,-0.56


# Data scheme #1

### input data
summary: last time step, human input to vehicle only:
- handwheelAngle
- throttle
- brake
- clutch

### output data / labels
summary: next time step, discrete derivative of motion signals:
- deriv_axCG
- deriv_ayCG
- deriv_pitchAngle
- deriv_pitchRate
- deriv_rollAngle
- deriv_rollRate
- deriv_vxCG
- deriv_vyCG
- deriv_wheelAccelFL
- deriv_wheelAccelFR
- deriv_wheelAccelRL
- deriv_wheelAccelRR
- deriv_yawAngle
- deriv_yawRate

### Objective
learn the incremental change in motion from last time step to next

In [9]:
# input
df_in = df[CS230.data.COLUMNS_HUMAN_INPUT]
df_in.head()

,brake,throttle,handwheelAngle
0,0.5,0.7,29.4
1,0.0,0.7,29.4
2,0.0,0.7,29.4
3,0.0,0.7,29.4
4,0.0,0.7,29.4


In [10]:
# output / labels
df_out = df[CS230.data.COLUMNS_DERIV]
df_out.head()

,deriv_axCG,deriv_ayCG,deriv_pitchAngle,deriv_pitchRate,deriv_rollAngle,deriv_rollRate,deriv_vxCG,deriv_vyCG,deriv_wheelAccelFL,deriv_wheelAccelFR,deriv_wheelAccelRL,deriv_wheelAccelRR,deriv_yawAngle,deriv_yawRate
0,0.00,0.00,0.0,0.00,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.00
1,0.08,-0.08,0.0,-0.29,0.0,1.19,0.0,-0.0,-0.10,0.1,0.2,0.07,0.0,-0.64
2,0.07,-0.10,0.0,-0.12,0.0,-0.76,0.0,0.0,0.29,0.0,-0.2,-0.05,0.0,-0.06
3,0.02,-0.07,0.0,0.12,0.0,0.30,-0.0,0.0,0.10,-0.2,0.1,-0.13,0.0,0.81
4,0.04,0.00,0.0,0.28,0.0,-0.26,0.0,0.0,-0.10,0.4,0.2,0.12,0.0,-0.56


# Data schema #1 - splitting into training / dev / test

Assume have data from 3 different vehicles, split into 3 different directories.

So will create 3 distinct training / test sets.

Plan for each file within a directory to be split:
- 90% train
- 5% dev
- 5% test

In [11]:
train_percent = 0.9
dev_percent = 0.05
test_percent = 0.05
        
file_path = file_paths[0]

df = CS230.data.load(file_path)
df = CS230.data.stride_rows(df, stride=10)
df = CS230.data.add_derivatives(df, stride=1)
df = CS230.data.clean_discontinuities(df, stride=1)

df_train, df_dev, df_test = CS230.data.get_data_sets(df, train_percent, dev_percent, test_percent, 
                                                     CS230.data.COLUMNS_HUMAN_INPUT, CS230.data.COLUMNS_DERIV)

In [12]:
print('train length: %s' % len(df_train))
df_train.head()

train length: 230760


,brake,throttle,handwheelAngle,deriv_axCG,deriv_ayCG,deriv_pitchAngle,deriv_pitchRate,deriv_rollAngle,deriv_rollRate,deriv_vxCG,deriv_vyCG,deriv_wheelAccelFL,deriv_wheelAccelFR,deriv_wheelAccelRL,deriv_wheelAccelRR,deriv_yawAngle,deriv_yawRate
0,0.0,0.7,-33.1,0.02,0.01,0.00,-0.02,0.00,0.01,0.00,-0.01,-0.10,-0.10,-0.49,-0.07,0.00,-0.28
1,0.0,49.1,-35.4,1.95,4.96,0.16,8.50,-0.09,-6.50,0.06,0.02,12.46,1.27,-4.61,0.99,-0.16,-0.86
2,0.0,84.8,4.9,-3.89,-3.26,-0.10,-6.23,0.07,10.35,0.02,-0.01,-7.75,2.45,15.50,-6.75,0.01,-0.02
3,0.0,0.8,-33.1,0.01,0.00,0.00,0.51,0.01,0.46,0.00,0.00,-0.29,0.20,-0.09,0.50,0.01,0.44
4,3.7,0.7,2.3,1.57,2.20,-0.01,-1.62,0.08,-2.74,-0.04,-0.08,2.64,-16.88,-25.02,-1.78,0.16,-0.29


In [13]:
print('dev length: %s' % len(df_dev))
df_dev.head()

dev length: 12820


,brake,throttle,handwheelAngle,deriv_axCG,deriv_ayCG,deriv_pitchAngle,deriv_pitchRate,deriv_rollAngle,deriv_rollRate,deriv_vxCG,deriv_vyCG,deriv_wheelAccelFL,deriv_wheelAccelFR,deriv_wheelAccelRL,deriv_wheelAccelRR,deriv_yawAngle,deriv_yawRate
0,0.0,6.0,92.9,5.49,-1.23,-0.14,-5.07,-0.15,-12.15,0.09,-0.05,-17.07,-1.57,43.56,7.14,0.46,0.57
1,0.0,84.8,6.4,-3.56,-2.26,-0.08,-2.58,-0.01,-2.84,0.01,-0.04,4.90,1.66,-2.75,18.85,0.02,-0.47
2,0.0,1.5,-1.8,0.22,-0.35,0.01,0.98,0.02,-0.49,0.00,-0.02,3.34,-1.77,0.29,-2.24,0.00,-0.17
3,20.0,0.7,7.2,-0.30,-1.42,0.00,1.74,-0.07,-10.83,-0.06,-0.01,-1.27,5.69,1.08,7.74,0.05,-0.31
4,0.0,37.1,0.7,1.45,-0.95,-0.03,-2.95,0.10,9.32,0.03,-0.01,-25.60,-1.08,-3.23,-0.14,0.00,-0.04


In [14]:
print('test length: %s' % len(df_test))
df_test.head()

test length: 12819


,brake,throttle,handwheelAngle,deriv_axCG,deriv_ayCG,deriv_pitchAngle,deriv_pitchRate,deriv_rollAngle,deriv_rollRate,deriv_vxCG,deriv_vyCG,deriv_wheelAccelFL,deriv_wheelAccelFR,deriv_wheelAccelRL,deriv_wheelAccelRR,deriv_yawAngle,deriv_yawRate
0,3.3,0.3,0.8,1.62,-0.86,-0.02,6.46,-0.12,-14.10,-0.04,-0.01,-4.42,-0.20,-7.16,-9.99,0.04,1.00
1,0.0,0.9,-7.3,0.00,-0.01,0.01,0.32,0.00,-0.06,0.00,0.00,0.29,0.30,-0.20,-0.13,0.00,-0.05
2,0.0,2.2,-12.2,-1.20,-0.08,0.01,1.08,0.00,-4.22,-0.01,0.00,-0.29,2.06,-1.08,-6.13,-0.02,0.54
3,0.0,1.4,-11.1,1.01,-1.54,0.01,3.78,0.02,1.26,-0.01,-0.01,5.79,3.34,5.89,-0.58,-0.12,0.97
4,0.0,64.2,-5.0,-3.31,3.45,-0.02,-2.54,-0.02,-2.83,0.00,0.02,10.40,4.51,-9.32,7.32,-0.03,-1.03


# Data schema #2

### input data
summary: from last time step, all data:
- HDOP
- PDOP
- PPS
- altitude
- axCG
- ayCG
- azCG
- brake
- chassisAccelFL
- chassisAccelFR
- chassisAccelRL
- chassisAccelRR
- clutch
- deflectionFL
- deflectionFR
- deflectionRL
- deflectionRR
- deriv_distance
- deriv_horizontalSpeed
- deriv_pitchAngle
- deriv_rollAngle
- deriv_vxCG
- deriv_vyCG
- deriv_yawAngle
- distance
- engineSpeed
- gpsOrientMode
- gpsPosMode
- gpsTime
- gpsVelMode
- handwheelAngle
- horizontalSpeed
- latitude
- longitude
- numSVsTracked
- orientAccuracy_heading
- orientAccuracy_pitch
- orientAccuracy_roll
- pitchAngle
- pitchRate
- posAccuracy_down
- posAccuracy_east
- posAccuracy_north
- rollAngle
- rollRate
- sideSlip
- throttle
- time
- vEast
- vNorth
- vUp
- velAccuracy_down
- velAccuracy_east
- velAccuracy_north
- vxCG
- vyCG
- vzCG
- wheelAccelFL
- wheelAccelFR
- wheelAccelRL
- wheelAccelRR
- yawAngle
- yawRate

### output data / label
summary: next time step, motion signals only:
- axCG
- ayCG
- pitchAngle
- pitchRate
- rollAngle
- rollRate
- vxCG
- vyCG
- wheelAccelFL
- wheelAccelFR
- wheelAccelRL
- wheelAccelRR
- yawAngle
- yawRate

### objective:
learn the absolute value of motion for next step (not change from prior step)

In [15]:
# input
df_in = df[CS230.data.COLUMNS_ALL]
df_in.head()

,HDOP,PDOP,PPS,altitude,axCG,ayCG,azCG,brake,chassisAccelFL,chassisAccelFR,...,velAccuracy_north,vxCG,vyCG,vzCG,wheelAccelFL,wheelAccelFR,wheelAccelRL,wheelAccelRR,yawAngle,yawRate
0,0.6,1.2,5.0,NaN,-0.28,0.57,9.78,0.5,9.506,9.771,...,0.0,-0.0,0.0,0.0,9.91,9.71,9.61,10.07,0.7,0.50
1,0.6,1.2,5.0,NaN,-0.20,0.49,9.79,0.0,9.526,9.751,...,0.0,0.0,-0.0,0.0,9.81,9.81,9.81,10.14,0.7,-0.14
2,0.6,1.2,5.0,NaN,-0.13,0.39,9.73,0.0,9.535,9.732,...,0.0,0.0,-0.0,0.0,10.10,9.81,9.61,10.09,0.7,-0.20
3,0.6,1.2,5.0,NaN,-0.11,0.32,9.76,0.0,9.535,9.732,...,0.0,-0.0,0.0,0.0,10.20,9.61,9.71,9.96,0.7,0.61
4,0.6,1.2,5.0,NaN,-0.07,0.32,9.75,0.0,9.506,9.722,...,0.0,-0.0,0.0,0.0,10.10,10.01,9.91,10.08,0.7,0.05


In [16]:
# output
df_out = df[CS230.data.COLUMNS_MOTION]
df_out.head()

,axCG,ayCG,pitchAngle,pitchRate,rollAngle,rollRate,vxCG,vyCG,wheelAccelFL,wheelAccelFR,wheelAccelRL,wheelAccelRR,yawAngle,yawRate
0,-0.28,0.57,0.4,0.30,-1.0,-0.55,-0.0,0.0,9.91,9.71,9.61,10.07,0.7,0.50
1,-0.20,0.49,0.4,0.01,-1.0,0.64,0.0,-0.0,9.81,9.81,9.81,10.14,0.7,-0.14
2,-0.13,0.39,0.4,-0.11,-1.0,-0.12,0.0,-0.0,10.10,9.81,9.61,10.09,0.7,-0.20
3,-0.11,0.32,0.4,0.01,-1.0,0.18,-0.0,0.0,10.20,9.61,9.71,9.96,0.7,0.61
4,-0.07,0.32,0.4,0.29,-1.0,-0.08,-0.0,0.0,10.10,10.01,9.91,10.08,0.7,0.05
